# UrbanSound8K Audio Classification - Exploratory Data Analysis

This notebook performs exploratory analysis on the UrbanSound8K dataset to understand:
- Dataset structure and class distribution
- Audio characteristics (duration, sampling rates)
- Sample visualizations (waveforms and spectrograms)

### Setup and Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import librosa
import librosa.display
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

### Configuration

In [ ]:
# Update this path to your UrbanSound8K dataset location
DATASET_PATH = "C:\\Users\\leono\\OneDrive\\Desktop\\AC2_folder\\UrbanSound8K"

### Load Metadata

In [ ]:
# Load the metadata CSV
dataset_path = Path(DATASET_PATH)
metadata_path = dataset_path / "metadata" / "UrbanSound8K.csv"

if not metadata_path.exists():
    print(f"ERROR: Metadata file not found at {metadata_path}")
    print("Please update DATASET_PATH in the cell above.")
else:
    df = pd.read_csv(metadata_path)
    print(f"Dataset loaded: {len(df)} audio samples")
    print(f"\nFirst few rows:")
    display(df.head())

### Dataset Overview

In [ ]:
#print("Dataset Information:")
print(f"Total samples: {len(df)}")
print(f"Number of classes: {df['classID'].nunique()}")
print(f"Number of folds: {df['fold'].nunique()}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nClass names: {sorted(df['class'].unique())}")

### Class Distribution

In [ ]:
# Count samples per class
class_counts = df['class'].value_counts().sort_index()
print("Samples per class:")
print(class_counts)

# Visualize class distribution
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Bar plot
class_counts.plot(kind='bar', ax=axes[0], color='skyblue', edgecolor='black')
axes[0].set_title('Class Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Sound Class', fontsize=12)
axes[0].set_ylabel('Number of Samples', fontsize=12)
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(axis='y', alpha=0.3)

# Pie chart
axes[1].pie(class_counts, labels=class_counts.index, autopct='%1.1f%%', startangle=90)
axes[1].set_title('Class Distribution (%)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

# Check if balanced
if class_counts.std() < 100:
    print("\n✓ Dataset is well-balanced across classes")
else:
    print("\n⚠ Dataset has class imbalance - consider class weights during training")

### Fold Distribution

In [ ]:
# Samples per fold
fold_counts = df['fold'].value_counts().sort_index()
print("Samples per fold:")
print(fold_counts)

# Visualize fold distribution
plt.figure(figsize=(10, 5))
fold_counts.plot(kind='bar', color='lightcoral', edgecolor='black')
plt.title('Fold Distribution', fontsize=14, fontweight='bold')
plt.xlabel('Fold Number', fontsize=12)
plt.ylabel('Number of Samples', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Cross-tabulation: class distribution per fold
print("\nClass distribution per fold:")
fold_class_dist = pd.crosstab(df['fold'], df['class'])
display(fold_class_dist)

### Audio File Analysis

In [ ]:
# Analyze a sample of audio files for duration and sampling rate
sample_size = min(100, len(df))  # Analyze 100 random files
sample_df = df.sample(n=sample_size, random_state=42)

durations = []
sampling_rates = []

print(f"Analyzing {sample_size} random audio files...")
for idx, row in sample_df.iterrows():
    audio_path = dataset_path / "audio" / f"fold{row['fold']}" / row['slice_file_name']
    
    if audio_path.exists():
        try:
            # Load audio to get duration and SR
            y, sr = librosa.load(audio_path, sr=None)
            duration = librosa.get_duration(y=y, sr=sr)
            
            durations.append(duration)
            sampling_rates.append(sr)
        except Exception as e:
            print(f"Error loading {audio_path}: {e}")

durations = np.array(durations)
sampling_rates = np.array(sampling_rates)

print(f"\nAnalysis complete: {len(durations)} files processed")

In [ ]:
# Duration statistics
print("Audio Duration Statistics:")
print(f"  Mean: {durations.mean():.2f} seconds")
print(f"  Std: {durations.std():.2f} seconds")
print(f"  Min: {durations.min():.2f} seconds")
print(f"  Max: {durations.max():.2f} seconds")

# Sampling rate statistics
print("\nSampling Rate Statistics:")
print(f"  Unique rates: {np.unique(sampling_rates)}")
print(f"  Most common: {np.bincount(sampling_rates.astype(int)).argmax()} Hz")

# Visualize distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Duration histogram
axes[0].hist(durations, bins=20, color='steelblue', edgecolor='black', alpha=0.7)
axes[0].axvline(durations.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {durations.mean():.2f}s')
axes[0].set_title('Audio Duration Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Duration (seconds)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].legend()
axes[0].grid(alpha=0.3)

# Sampling rate distribution
sr_counts = pd.Series(sampling_rates).value_counts().sort_index()
sr_counts.plot(kind='bar', ax=axes[1], color='coral', edgecolor='black')
axes[1].set_title('Sampling Rate Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Sampling Rate (Hz)', fontsize=12)
axes[1].set_ylabel('Number of Files', fontsize=12)
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### Audio Visualization

In [1]:
def plot_audio_features(audio_path, class_name):
    """Plot waveform, spectrogram, and mel spectrogram for an audio file"""
    # Load audio
    y, sr = librosa.load(audio_path, sr=22050)
    
    # Create figure
    fig, axes = plt.subplots(3, 1, figsize=(14, 10))
    fig.suptitle(f'Audio Features: {class_name}', fontsize=16, fontweight='bold')
    
    # Waveform
    librosa.display.waveshow(y, sr=sr, ax=axes[0], color='blue')
    axes[0].set_title('Waveform', fontsize=12)
    axes[0].set_xlabel('Time (s)')
    axes[0].set_ylabel('Amplitude')
    axes[0].grid(alpha=0.3)
    
    # Spectrogram
    D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)
    img = librosa.display.specshow(D, sr=sr, x_axis='time', y_axis='hz', ax=axes[1], cmap='viridis')
    axes[1].set_title('Spectrogram', fontsize=12)
    axes[1].set_ylabel('Frequency (Hz)')
    fig.colorbar(img, ax=axes[1], format='%+2.0f dB')
    
    # Mel Spectrogram
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    img = librosa.display.specshow(mel_spec_db, sr=sr, x_axis='time', y_axis='mel', ax=axes[2], cmap='magma')
    axes[2].set_title('Mel Spectrogram', fontsize=12)
    axes[2].set_xlabel('Time (s)')
    axes[2].set_ylabel('Mel Frequency')
    fig.colorbar(img, ax=axes[2], format='%+2.0f dB')
    
    plt.tight_layout()
    plt.show()

In [2]:
# Visualize one sample from each class
print("Visualizing sample audio files from each class...\n")

for class_name in sorted(df['class'].unique()):
    # Get one sample from this class
    sample = df[df['class'] == class_name].sample(n=1, random_state=42).iloc[0]
    audio_path = dataset_path / "audio" / f"fold{sample['fold']}" / sample['slice_file_name']
    
    if audio_path.exists():
        print(f"Plotting: {class_name} - {sample['slice_file_name']}")
        plot_audio_features(audio_path, class_name)
    else:
        print(f"File not found: {audio_path}")

Visualizing sample audio files from each class...



NameError: name 'df' is not defined

### MFCC Visualization

In [4]:
# Extract and visualize MFCCs for sample files
print("Extracting MFCC features for sample files...\n")

fig, axes = plt.subplots(2, 5, figsize=(20, 8))
fig.suptitle('MFCC Features by Class', fontsize=16, fontweight='bold')
axes = axes.ravel()

for idx, class_name in enumerate(sorted(df['class'].unique())):
    # Get one sample from this class
    sample = df[df['class'] == class_name].sample(n=1, random_state=42).iloc[0]
    audio_path = dataset_path / "audio" / f"fold{sample['fold']}" / sample['slice_file_name']
    
    if audio_path.exists():
        # Load and extract MFCCs
        y, sr = librosa.load(audio_path, sr=22050)
        mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=40)
        
        # Plot
        img = librosa.display.specshow(mfccs, x_axis='time', ax=axes[idx], cmap='coolwarm')
        axes[idx].set_title(class_name, fontsize=10)
        axes[idx].set_ylabel('MFCC Coefficients')
        
plt.tight_layout()
plt.show()

Extracting MFCC features for sample files...



NameError: name 'plt' is not defined